In [ ]:
### 뚝섬역 근처 top5 아파트 시세
# 중앙하이츠빌, 뚝섬현대, 대우2차, 쌍용, 서울숲한양현대

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

In [4]:
## 복호화함수
import base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad



def decrypt(encrypted_text, secret):
    """
    AES ECB 모드로 암호화된 텍스트를 복호화합니다.
    
    Args:
        encrypted_text (str): Base64로 인코딩된 암호화된 텍스트
    
    Returns:
        str: 복호화된 평문 텍스트
    """
    # Base64 디코딩
    encrypted_bytes = base64.b64decode(encrypted_text)
    
    # 키 생성
    key = get_key(secret)
    
    # AES 복호화 객체 생성 (ECB 모드)
    cipher = AES.new(key, AES.MODE_ECB)
    
    # 복호화
    decrypted_bytes = cipher.decrypt(encrypted_bytes)
    
    # PKCS7 패딩 제거
    unpadded_bytes = unpad(decrypted_bytes, AES.block_size)
    
    # UTF-8 문자열로 변환
    return unpadded_bytes.decode('utf-8')

def get_key(secret):
    """
    비밀 문자열을 AES 키로 변환합니다.
    키 길이에 따라 적절한 크기로 조정합니다.
    
    Args:
        secret (str): 비밀 문자열
    
    Returns:
        bytes: AES 키 (16, 24, 또는 32 바이트)
    """
    # 문자열을 UTF-8 바이트로 변환
    key_bytes = secret.encode('utf-8')
    length = len(key_bytes)
    
    # 키 길이에 따라 조정
    if length < 16:
        # 16바이트보다 작으면 16바이트로 패딩
        adjusted_key = key_bytes[:16].ljust(16, b'\x00')
    elif length > 16 and length < 24:
        # 16바이트보다 크고 24바이트보다 작으면 24바이트로 조정
        adjusted_key = key_bytes[:24].ljust(24, b'\x00')
    elif length > 24 and length < 32:
        # 24바이트보다 크고 32바이트보다 작으면 32바이트로 조정
        adjusted_key = key_bytes[:32].ljust(32, b'\x00')
    elif length >= 32:
        # 32바이트 이상이면 32바이트로 잘라내기
        adjusted_key = key_bytes[:32]
    else:
        # 정확히 16바이트면 그대로 사용
        adjusted_key = key_bytes
    
    return adjusted_key

In [5]:
# 억단위 숫자로
import re

def parse_korean_price(text):
    """
    '1억 2,300' → 123000000 변환
    """
    if pd.isna(text):
        return None
    text = text.replace(',', '').replace(' ', '')
    total = 0
    match = re.match(r'(?:(\d+)억)?(?:(\d+)천)?(\d+)?', text)
    if not match:
        return None
    if match.group(1):  # 억
        total += int(match.group(1)) * 100_000_000
    if match.group(2):  # 천만 단위
        total += int(match.group(2)) * 10_000_000
    if match.group(3):  # 만 단위 이하
        total += int(match.group(3))
    return total

In [3]:
## 뚝섬역 중앙하이츠빌
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=51194"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['75436954131569951194']


In [10]:
## 뚝섬역 중앙하이츠빌
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=51194&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_1 = pd.DataFrame(rows)

key = '75436954131569951194'

# 가격과 전월세 컬럼 복호화
df_1['가격'] = df_1['가격'].apply(lambda x: decrypt(x, key))
df_1['전월세'] = df_1['전월세'].apply(lambda x: decrypt(x, key))

df_1['가격(숫자)'] = df_1['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_1

,거래일,층수,가격,전월세,가격(숫자)
0,2025.06.25,10,"17억 1,000",,1700001000
1,2025.06.21,6,"17억 8,000",,1700008000
2,2025.06.17,9,"17억 6,000",,1700006000
3,2025.05.23,13,"17억 5,000",,1700005000
4,2025.05.15,3,"15억 5,000",,1500005000
...,...,...,...,...,...
199,2006.10.22,2,"5억 4,000",,500004000
200,2006.09.10,21,"6억 1,000",,600001000
201,2006.06.01,23,"5억 5,000",,500005000
202,2006.04.16,15,"5억 5,500",,500005500


In [11]:
## 뚝섬역 뚝섬현대
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=265"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['755369543200072265']


In [13]:
## 뚝섬역 뚝섬현대
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=265&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=10000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_2 = pd.DataFrame(rows)

key = '755369543200072265'

# 가격과 전월세 컬럼 복호화
df_2['가격'] = df_2['가격'].apply(lambda x: decrypt(x, key))
df_2['전월세'] = df_2['전월세'].apply(lambda x: decrypt(x, key))

df_2['가격(숫자)'] = df_2['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_2

,거래일,층수,가격,전월세,가격(숫자)
0,2025.07.12,10,"12억 7,000",,1200007000
1,2025.07.12,10,"12억 7,000",,1200007000
2,2025.06.11,13,"12억 4,500",,1200004500
3,2025.06.09,4,12억,,1200000000
4,2025.02.20,1,"11억 2,000",,1100002000
...,...,...,...,...,...
76,2006.11.05,4,"3억 8,000",,300008000
77,2006.11.03,3,"3억 8,000",,300008000
78,2006.05.15,11,"3억 8,000",,300008000
79,2006.05.02,7,"3억 8,000",,300008000


In [14]:
## 뚝섬역 대우2차
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=262"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['752369543200183262']


In [15]:
# 뚝섬역 대우2차
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=262&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_3 = pd.DataFrame(rows)

key = '752369543200183262'

# 가격과 전월세 컬럼 복호화
df_3['가격'] = df_3['가격'].apply(lambda x: decrypt(x, key))
df_3['전월세'] = df_3['전월세'].apply(lambda x: decrypt(x, key))

df_3['가격(숫자)'] = df_3['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_3

,거래일,층수,가격,전월세,가격(숫자)
0,2025.07.14,3,14억,,1400000000
1,2025.06.27,15,"14억 3,000",,1400003000
2,2025.06.26,1,"12억 4,500",,1200004500
3,2025.06.25,1,"12억 8,000",,1200008000
4,2025.06.25,7,"14억 2,000",,1400002000
...,...,...,...,...,...
215,2006.05.01,12,"4억 2,300",,400002300
216,2006.04.09,18,"4억 5,300",,400005300
217,2006.03.19,12,"4억 2,000",,400002000
218,2006.02.23,9,"3억 9,700",,300009700


In [16]:
# 뚝섬역 쌍용
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=267"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)

['757369543199998267']


In [17]:
# 뚝섬역 씽용
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=267&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=10000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_4 = pd.DataFrame(rows)

key = '757369543199998267'

# 가격과 전월세 컬럼 복호화
df_4['가격'] = df_4['가격'].apply(lambda x: decrypt(x, key))
df_4['전월세'] = df_4['전월세'].apply(lambda x: decrypt(x, key))

df_4['가격(숫자)'] = df_4['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_4

,거래일,층수,가격,전월세,가격(숫자)
0,2025.06.27,3,"16억 8,000",,1600008000
1,2025.06.22,6,"16억 2,000",,1600002000
2,2025.06.20,11,"16억 6,500",,1600006500
3,2025.06.20,4,14억,,1400000000
4,2025.06.16,2,"13억 4,500",,1300004500
...,...,...,...,...,...
778,2006.03.16,13,3억,,300000000
779,2006.03.15,15,5억,,500000000
780,2006.03.15,13,"3억 2,800",,300002800
781,2006.03.09,18,"4억 8,500",,400008500


In [18]:
## 뚝섬역 서울숲한양현대
import requests
import re

# apt_info.jsp
url = "https://asil.kr/app/apt_info.jsp?os=pc&apt=270"
headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
}
response = requests.get(url, headers=headers)

# HTML 본문
html = response.text

# 정규표현식으로 getKey("숫자")에서 숫자만 문자열로 추출
keys = re.findall(r'getKey\("(\d+)"\)', html)

# 결과 확인
print(keys)


['750369543199887270']


In [20]:
# 뚝섬역 서울숲한양현대
import requests
import pandas as pd


url = ("https://asil.kr/app/data/apt_price_m2_newver_6.jsp?sido=11&dealmode=1&building=apt&seq=270&m2=0&py=&py_type=&isPyQuery=false&year=9999&u=0&start=0&count=1000&dong_name=&order=")

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://asil.kr/",
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

try:
    new_data = response.json()
except ValueError:
    print("JSON 디코딩 실패. 응답 내용:")
    print(response.text)
    exit()

rows = []

if new_data and "val" in new_data[0]:
    for monthly in new_data[0]["val"]:
        yyyymm = monthly["yyyymm"]
        for daily in monthly["val"]:
            day = daily["day"]
            for deal in daily["val"]:
                rows.append({
                    "거래일": f"{yyyymm[:4]}.{yyyymm[4:]}.{day:02}",
                    "층수": deal.get("floor"),
                    "가격": deal.get("money"),
                    "전월세": deal.get("rent")
                })
else:
    print("빈 데이터 혹은 예상 구조와 다름")
    exit()

df_5 = pd.DataFrame(rows)

key = '750369543199887270'

# 가격과 전월세 컬럼 복호화
df_5['가격'] = df_5['가격'].apply(lambda x: decrypt(x, key))
df_5['전월세'] = df_5['전월세'].apply(lambda x: decrypt(x, key))

df_5['가격(숫자)'] = df_5['가격'].apply(parse_korean_price)  # parse_korean_price는 이전에 정의한 함수
df_5

,거래일,층수,가격,전월세,가격(숫자)
0,2025.07.01,3,"13억 8,000",,1300008000
1,2025.06.18,2,"11억 1,900",,1100001900
2,2025.06.18,5,"11억 2,000",,1100002000
3,2025.06.07,10,"11억 3,500",,1100003500
4,2025.06.06,2,"13억 6,500",,1300006500
...,...,...,...,...,...
130,2006.10.16,4,3억,,300000000
131,2006.09.09,12,"3억 2,000",,300002000
132,2006.08.29,10,"2억 9,800",,200009800
133,2006.06.12,8,"2억 9,000",,200009000
